# PS6
Chang Gao

Note: The code is written with the help of paid version of claude.ai and mostly, in node.js. Python cannot deal with some clear issues successfully. Some of the comments are in Chinese, which I do not have sufficient time to translate. I cannot remember all the steps I did in working on my research, and I did't run the code in this Jupyter Notebook.

## Q3 Data cleaning
Here is how I cleaned the monthly housing index data scraped from China's National Bureau of Statistics.

Both node.js and Python is used for data cleaning.

### Step 1 & 2: Merge some split CSVs, Remove extra CSVs
One issue with the scrapped data is that, for house of different size, for some month, the table in htmls are split. Becasue the table is too long, they are split into two halfs. Fortunately, the split method is consistent across time. I addressed this issue by let cluade write code on: for each CSV, check if name of "specific_city_1" exist in the table, and if name of "specific_city_2" exist in the table, if not, merge.

For each month, the number of csv scrapped from these htmls are around 4-12, I checked their details and found that the first few csv conatins sufficient data. The format of csv and htmls differs across years. For each month I have a folder that contain these csv. I let claude worte a code that can read the first few rows (and second column since they are housing index values) of the data, and remove the csv if same data is founded. 

After this step, I have 4-5 csv left in each "month" folder.

In [ ]:
%%javascript
const fs = require('fs');
const { JSDOM } = require('jsdom');
const path = require('path');

// 检查表格是否包含特定城市
function tableContainsCity(table, cityName) {
    return table.textContent.replace(/\s+/g, '').includes(cityName);
}

// 获取表格的特征值（用于查重）
function getTableSignature(tableData) {
    if (tableData.length < 15) {
        console.log('表格行数不足15行，无法获取特征值');
        return null;
    }

    // 提取第二列第10-15行的数值
    const values = [];
    for (let i = 10; i <= 15; i++) {
        if (tableData[i] && tableData[i][1]) {
            // 提取数字部分
            const numStr = tableData[i][1].replace(/[^\d.]/g, '');
            if (numStr) {
                values.push(numStr);
            }
        }
    }

    if (values.length > 0) {
        // 将所有数值连接成一个字符串作为特征值
        const signature = values.join('|');
        console.log('提取的特征值:', signature);
        return signature;
    }
    
    console.log('无法从指定行范围获取有效数值');
    return null;
}

// 提取表格数据
function extractTableData(table) {
    const data = [];
    const rows = table.querySelectorAll('tr');
    
    rows.forEach(row => {
        const cells = row.querySelectorAll('td, th');
        const rowData = Array.from(cells).map(cell => {
            return cell.textContent.replace(/\s+/g, ' ').trim();
        });
        
        if (rowData.length > 0) {
            data.push(rowData);
        }
    });
    return data;
}

// 处理单个HTML文件
function processHtmlFile(filePath) {
    try {
        const fileName = path.basename(filePath);
        const match = fileName.match(/^(\d{4})(\d{2})\.html$/);
        if (!match) {
            console.log(`跳过不符合命名规则的文件: ${fileName}`);
            return;
        }

        const year = match[1];
        const month = match[2];
        console.log(`\n处理文件: ${fileName} (${year}年${month}月)`);

        const html = fs.readFileSync(filePath, 'utf8');
        const dom = new JSDOM(html);
        const document = dom.window.document;

        const allTables = Array.from(document.querySelectorAll('table'));
        const processedTables = [];
        const signatures = new Set(); // 用于存储已处理的表格特征值

        // 遍历表格查找包含呼和浩特的表格
        for (let i = 0; i < allTables.length; i++) {
            const currentTable = allTables[i];
            if (tableContainsCity(currentTable, '呼和浩特')) {
                const currentTableData = extractTableData(currentTable);
                let finalTableData = currentTableData;
                
                // 检查当前表格是否包含平顶山
                if (!tableContainsCity(currentTable, '平顶山')) {
                    // 查找下一个表格
                    const nextTable = allTables[i + 1];
                    if (nextTable && tableContainsCity(nextTable, '平顶山')) {
                        console.log('找到配对的表格，准备合并数据');
                        const nextTableData = extractTableData(nextTable);
                        
                        // 合并两个表格的数据（保留表头）
                        finalTableData = [
                            ...currentTableData.slice(0, 1), // 表头
                            ...currentTableData.slice(1),    // 第一个表格数据
                            ...nextTableData.slice(1)        // 第二个表格数据
                        ];
                        
                        // 跳过下一个表格
                        i++;
                    }
                }

                // 检查是否重复
                const signature = getTableSignature(finalTableData);
                if (signature && !signatures.has(signature)) {
                    signatures.add(signature);
                    processedTables.push(finalTableData);
                    console.log(`添加新表格，特征值: ${signature}`);
                } else if (signature) {
                    console.log(`跳过重复表格，特征值: ${signature}`);
                } else {
                    console.log('表格无法获取特征值，仍然保留');
                    processedTables.push(finalTableData);
                }
            }
        }

        // 创建年份目录
        const yearDir = path.join('./output', year);
        if (!fs.existsSync(yearDir)) {
            fs.mkdirSync(yearDir, { recursive: true });
        }

        // 保存处理后的表格
        processedTables.forEach((tableData, index) => {
            if (tableData.length > 0) {
                const outputFileName = path.join(yearDir, `${year}${month}_table_${index + 1}.csv`);
                const csvContent = tableData.map(row => {
                    return row.map(cell => `"${cell.replace(/"/g, '""')}"`).join(',');
                }).join('\n');
                
                fs.writeFileSync(outputFileName, '\ufeff' + csvContent, 'utf8');
                console.log(`表格数据已保存到: ${outputFileName}`);
            }
        });

        return {
            fileName,
            year,
            month,
            tablesFound: processedTables.length
        };
        
    } catch (error) {
        console.error(`处理文件 ${filePath} 时出错:`, error);
        return {
            fileName: path.basename(filePath),
            error: error.message
        };
    }
}

// 按年份范围处理文件
function processFilesByYearRange(startYear, endYear) {
    console.log(`开始处理 ${startYear} 到 ${endYear} 年的数据`);
    
    // 创建输出根目录
    const outputDir = './output';
    if (!fs.existsSync(outputDir)) {
        fs.mkdirSync(outputDir);
    }

    // 读取并筛选指定年份范围的文件
    const files = fs.readdirSync('.');
    const htmlFiles = files.filter(file => {
        const match = file.match(/^(\d{4})(\d{2})\.html$/);
        if (!match) return false;
        const year = parseInt(match[1]);
        return year >= startYear && year <= endYear;
    }).sort();
    
    if (htmlFiles.length === 0) {
        console.log(`未找到 ${startYear}-${endYear} 年范围内的HTML文件`);
        return;
    }

    console.log(`找到 ${htmlFiles.length} 个文件待处理：`);
    htmlFiles.forEach(file => console.log(`- ${file}`));
    
    // 处理文件并收集结果
    const results = htmlFiles.map(file => processHtmlFile(file));

    // 生成处理报告
    const successfulFiles = results.filter(r => r && !r.error);
    const failedFiles = results.filter(r => r && r.error);
    
    const reportContent = `处理报告 (${startYear}-${endYear})
生成时间: ${new Date().toLocaleString()}
总文件数: ${htmlFiles.length}
成功处理: ${successfulFiles.length}
处理失败: ${failedFiles.length}

成功处理的文件:
${successfulFiles.map(r => `- ${r.fileName}: ${r.year}年${r.month}月, 找到 ${r.tablesFound} 个表格`).join('\n')}

${failedFiles.length > 0 ? `处理失败的文件:
${failedFiles.map(r => `- ${r.fileName}: ${r.error}`).join('\n')}` : ''}
`;

    const reportFileName = path.join(outputDir, `处理报告_${startYear}-${endYear}.txt`);
    fs.writeFileSync(reportFileName, reportContent, 'utf8');
    console.log(`\n${startYear}-${endYear}年数据处理完成！详细报告已保存到 ${reportFileName}`);
}

// 使用命令行参数获取年份范围
const args = process.argv.slice(2);
const startYear = parseInt(args[0]) || 2011;
const endYear = parseInt(args[1]) || 2013;

// 运行处理程序
processFilesByYearRange(startYear, endYear);

### Step 3: Check and keep the sufficient 4 CSVs, rename
The sufficient 4 CSVs (xxxx_table_2 to xxxx_table_5) are: 1-new resendial housing price; 2-used resendial housing price; 3-new,small,medium,large; 4-used,small,medium,large.
Some of the folders contains a fifth CSV (table_1) of some summary. I did the remove of table_1 and rename table_2 as new table_1, table_3 as new table_2...

In [ ]:
import os
from datetime import datetime, timedelta

def process_all_years(base_path, start_year=2017, end_year=2017):
    """
    处理指定路径下的所有年份文件
    
    参数:
    base_path: 数据文件夹的完整路径，包含所有年份子文件夹的目录
    start_year: 开始年份
    end_year: 结束年份
    """
    def get_month_range(start_year, end_year):
        start_date = datetime(start_year, 1, 1)
        end_date = datetime(end_year, 12, 31)
        current_date = start_date
        months = []
        
        while current_date <= end_date:
            months.append(current_date.strftime("%Y%m"))
            if current_date.month == 12:
                current_date = datetime(current_date.year + 1, 1, 1)
            else:
                current_date = datetime(current_date.year, current_date.month + 1, 1)
        return months

    def rename_table_files(year_folder, year_month):
        # 首先删除table_1
        table1_path = os.path.join(year_folder, f"{year_month}_table_1.csv")
        if os.path.exists(table1_path):
            try:
                os.remove(table1_path)
                print(f"已删除: {table1_path}")
            except Exception as e:
                print(f"删除文件失败 {table1_path}: {str(e)}")
        
        # 重命名其他文件
        for i in range(2, 6):
            old_file = os.path.join(year_folder, f"{year_month}_table_{i}.csv")
            new_file = os.path.join(year_folder, f"{year_month}_table_{i-1}.csv")
            if os.path.exists(old_file):
                try:
                    os.rename(old_file, new_file)
                    print(f"已重命名: {old_file} -> {new_file}")
                except Exception as e:
                    print(f"重命名失败 {old_file}: {str(e)}")

    # 获取所有需要处理的月份
    all_months = get_month_range(start_year, end_year)
    
    # 处理每个月份
    for year_month in all_months:
        year = year_month[:4]
        year_folder = os.path.join(base_path, year)
        
        if os.path.exists(year_folder):
            print(f"\n处理 {year_month} 的文件...")
            rename_table_files(year_folder, year_month)
        else:
            print(f"警告: 文件夹 {year_folder} 不存在，跳过...")

# 使用示例：
# 替换为你的实际路径
base_path = "/Users/ChangGao/house-price-analysis/outputcopy"  # 替换这个路径
process_all_years(base_path)

### Step 4: Clear each CSV
For each CSV, I check where its first row is variable name or the title. I removed the title and variable names.

Another issue is that the data of 70 cities are put into two columns, with 35 cities in each column, I merged the seperated columns.

I also removed the year-to-year and month-to-month ratio from the original data set as the monthly price indicies have been sufficient.

In [ ]:
%%javascript
import fs from 'fs/promises';
import path from 'path';
import Papa from 'papaparse';

/**
 * 递归获取目录下的所有文件
 */
async function getAllFiles(dirPath) {
    const files = await fs.readdir(dirPath);
    const allFiles = [];
    
    for (const file of files) {
        const fullPath = path.join(dirPath, file);
        const stat = await fs.stat(fullPath);
        
        if (stat.isDirectory()) {
            console.log(`正在搜索子目录: ${file}`);
            const subFiles = await getAllFiles(fullPath);
            allFiles.push(...subFiles);
        } else {
            allFiles.push({
                name: file,
                path: fullPath
            });
        }
    }
    
    return allFiles;
}

/**
 * 获取所有表格3和表格4的CSV文件
 */
async function getTargetFiles(directoryPath) {
    try {
        console.log('正在检查目录:', directoryPath);
        
        const allFiles = await getAllFiles(directoryPath);
        const targetFiles = allFiles.filter(file => {
            const isMatch = /^\d{6}_table_[34]\.csv$/.test(file.name);
            console.log(`检查文件 ${file.path}: ${isMatch ? '符合条件' : '不符合条件'}`);
            return isMatch;
        });
        
        return targetFiles;
    } catch (error) {
        console.error('读取目录失败:', error);
        return [];
    }
}

/**
 * 处理单个CSV文件
 */
async function processCSV(filepath) {
    try {
        console.log(`\n开始处理文件: ${filepath}`);
        
        const fileContent = await fs.readFile(filepath, { encoding: 'utf-8' });
        const parsed = Papa.parse(fileContent, {
            skipEmptyLines: true,
            encoding: 'utf-8'
        });
        
        const rows = parsed.data;
        let yearOnYearRowIndex = -1;
        let headerRowIndex = -1;
        let beijingRowIndex = -1;
        
        // 找到表头、同比行和北京数据行
        for (let i = 0; i < rows.length; i++) {
            if (rows[i].includes('城市') && !rows[i].join('').includes('七十个大中城市')) {
                headerRowIndex = i;
            }
            if (rows[i].join('').includes('同比')) {
                yearOnYearRowIndex = i;
            }
            // 检查是否为北京数据行
            if (rows[i][0] && rows[i][0].trim().replace(/\s+/g, '') === '北京') {
                beijingRowIndex = i;
                console.log(`找到北京数据行，索引为: ${i}`);
            }
        }
        
        if (yearOnYearRowIndex === -1 || headerRowIndex === -1 || beijingRowIndex === -1) {
            throw new Error('未找到必要的数据行');
        }
        
        const headers = rows[headerRowIndex].filter(col => col !== '');
        const processedData = [];
        processedData.push(headers);
        
        // 从北京数据行开始处理
        let currentRow = beijingRowIndex;
        while (currentRow < rows.length) {
            const row = rows[currentRow];
            if (row.length === 0 || !row[0]) {
                currentRow++;
                continue;
            }
            
            const cityName = row[0].trim();
            const yearOnYearData = [];
            
            for (let j = 1; j < headers.length; j++) {
                const colIndex = (j - 1) * 3 + 1;
                yearOnYearData.push(row[colIndex]);
            }
            
            processedData.push([cityName, ...yearOnYearData]);
            currentRow++;
        }
        
        // 添加 BOM 以确保 Excel 正确识别中文
        const BOM = '\ufeff';
        const processedCSV = BOM + Papa.unparse(processedData, {
            encoding: 'utf-8'
        });
        
        const filename = path.basename(filepath);
        const match = filename.match(/(\d{6})_table_(\d+)\.csv$/);
        if (!match) {
            throw new Error('文件名格式不正确');
        }
        const [, yyyymm, tableNum] = match;
        const newFilename = `processed_${yyyymm}_table_${tableNum}.csv`;
        const outputPath = path.join(path.dirname(filepath), newFilename);
        
        return {
            data: processedCSV,
            filename: newFilename,
            outputPath: outputPath
        };
    } catch (error) {
        throw error;
    }
}

/**
 * 批量处理CSV文件
 */
async function batchProcessCSVFiles(directoryPath) {
    try {
        console.log('开始批量处理CSV文件...\n');
        
        const targetFiles = await getTargetFiles(directoryPath);
        
        if (targetFiles.length === 0) {
            console.log('未找到表格3或表格4的CSV文件');
            return;
        }
        
        console.log(`\n找到 ${targetFiles.length} 个文件需要处理:`);
        targetFiles.forEach(file => console.log(`- ${file.path}`));
        
        for (const file of targetFiles) {
            try {
                const result = await processCSV(file.path);
                await fs.writeFile(result.outputPath, result.data, { encoding: 'utf-8' });
                
                console.log(`\n成功处理文件: ${file.path}`);
                console.log(`已保存为: ${result.outputPath}`);
            } catch (error) {
                console.error(`处理文件 ${file.path} 失败:`, error.message);
            }
        }
        
        console.log('\n所有文件处理完成');
    } catch (error) {
        console.error('批量处理失败:', error);
    }
}

// 获取命令行参数
const args = process.argv.slice(2);
if (args.length < 2 || args[0] !== 'dir') {
    console.log('使用方法: node process_34.js dir <目录路径>');
    process.exit(1);
}

const directoryPath = args[1];
batchProcessCSVFiles(directoryPath);

In [ ]:
%%javascript
const fs = require('fs').promises;
const path = require('path');
const Papa = require('papaparse');

async function processCSV(filepath) {
    try {
        // 读取CSV文件并添加BOM
        const content = await fs.readFile(filepath);
        const decoder = new TextDecoder('utf-8');
        const text = decoder.decode(content);
        
        // 解析CSV
        const parseResult = Papa.parse(text, {
            header: false,
            skipEmptyLines: true,
            encoding: 'utf-8'
        });
        
        let data = parseResult.data;
        
        // 查找"乌鲁木齐"所在的行
        let urumqiIndex = -1;
        for (let i = 0; i < data.length; i++) {
            if (data[i].some(cell => typeof cell === 'string' && cell.replace(/\s+/g, '') === '乌鲁木齐')) {
                urumqiIndex = i;
                break;
            }
        }
        
        if (urumqiIndex === -1) {
            console.log('未找到乌鲁木齐');
            return;
        }
        
        // 删除乌鲁木齐后一行的内容（如果有的话）
        if (urumqiIndex + 1 < data.length) {
            data.splice(urumqiIndex + 1, 1);
        }
        
        // 查找"唐山"所在的列
        let tangShanColIndex = -1;
        for (let i = 0; i < data[0].length; i++) {
            const found = data.some(row => 
                typeof row[i] === 'string' && row[i].replace(/\s+/g, '') === '唐山'
            );
            if (found) {
                tangShanColIndex = i;
                break;
            }
        }
        
        if (tangShanColIndex === -1) {
            console.log('未找到唐山');
            return;
        }
        
        // 提取需要移动的列
        const columnsToMove = data.map(row => row.slice(tangShanColIndex));
        const remainingColumns = data.map(row => row.slice(0, tangShanColIndex));
        
        // 构建新的数据结构
        const newData = [];
        
        // 添加乌鲁木齐之前的行
        for (let i = 0; i <= urumqiIndex; i++) {
            newData.push(remainingColumns[i]);
        }
        
        // 添加移动的列数据
        for (let i = 0; i < data.length; i++) {
            newData.push(columnsToMove[i]);
        }

        // 新增：只保留每行的前两列
        const trimmedData = newData.map(row => {
            if (row.length >= 2) {
                return row.slice(0, 2);
            }
            // 如果行长度不足2，则保持原样
            return row;
        });
        
        // 将数据转换回CSV格式
        const csv = Papa.unparse(trimmedData, {
            encoding: "utf8"
        });
        
        // 添加 BOM 标记
        const BOM = '\ufeff';
        const csvWithBOM = BOM + csv;
        
        // 构建新的文件名
        const dir = path.dirname(filepath);
        const filename = path.basename(filepath);
        const newFilename = filename.replace('processed_', 'processed2_');
        const newFilepath = path.join(dir, newFilename);
        
        // 保存文件，确保使用 UTF-8 with BOM
        await fs.writeFile(newFilepath, csvWithBOM, { encoding: 'utf8' });
        console.log(`已处理并保存到: ${newFilepath}`);
        
    } catch (error) {
        console.error('处理文件时发生错误:', error);
    }
}

async function processAllFiles(baseDir) {
    try {
        // 读取所有年份目录
        const years = await fs.readdir(baseDir);
        
        for (const year of years) {
            const yearPath = path.join(baseDir, year);
            const stat = await fs.stat(yearPath);
            
            if (stat.isDirectory()) {
                // 读取年份目录中的文件
                const files = await fs.readdir(yearPath);
                
                for (const file of files) {
                    if (file.match(/^processed_.*_table_[12]\.csv$/)) {
                        const filepath = path.join(yearPath, file);
                        console.log(`正在处理文件: ${filepath}`);
                        await processCSV(filepath);
                    }
                }
            }
        }
        
    } catch (error) {
        console.error('处理文件夹时发生错误:', error);
    }
}

// 检查命令行参数
const args = process.argv.slice(2);
if (args.length === 0) {
    console.log('请提供文件路径或目录路径');
    console.log('使用方法:');
    console.log('处理单个文件: node script.js file <filepath>');
    console.log('处理整个目录: node script.js dir <dirpath>');
    process.exit(1);
}

const mode = args[0];
const inputPath = args[1];

if (!inputPath) {
    console.log('请提供路径');
    process.exit(1);
}

if (mode === 'file') {
    processCSV(inputPath);
} else if (mode === 'dir') {
    processAllFiles(inputPath);
} else {
    console.log('无效的模式，请使用 "file" 或 "dir"');
    process.exit(1);
}

### Step 5: Merge all the CSVs
The difficulty now comes from city names, such as "Beijing" in Chinese it is "北京",  but in these CSVs, it could be "北 京" or "北京*"

In [ ]:
%%javascript
import fs from 'fs/promises';
import path from 'path';
import { fileURLToPath } from 'url';
import Papa from 'papaparse';

process.stdout.setEncoding('utf8');

const args = process.argv.slice(2);
const dirIndex = args.indexOf('dir');
const dirPath = dirIndex !== -1 ? args[dirIndex + 1] : '.';

// 城市名称映射表
const CITY_NAME_MAPPING = {
    '襄樊': '襄阳', // 将襄樊映射到襄阳
    // 如果将来发现其他更名的城市，可以在这里添加
};

// 城市名称标准化函数
function standardizeCityName(cityName) {
    if (cityName === '城市' || !cityName) return null;
    
    // 标准化处理
    let standardName = cityName
        .replace(/\s+/g, '')
        .replace(/\*/g, '')
        .replace(/［/g, '[')
        .replace(/］/g, ']')
        .trim();
    
    // 应用城市名称映射
    return CITY_NAME_MAPPING[standardName] || standardName;
}

async function readCSVFile(filePath, tableType) {
    try {
        const content = await fs.readFile(filePath, 'utf8');
        return new Promise((resolve, reject) => {
            const parseConfig = {
                skipEmptyLines: true,
                encoding: 'utf8',
                error: error => reject(error)
            };

            if (tableType === '1' || tableType === '2') {
                parseConfig.header = false;
                parseConfig.complete = results => {
                    const processedData = results.data
                        .map(row => {
                            const cityName = standardizeCityName(row[0]);
                            if (!cityName) return null;
                            return {
                                city: cityName,
                                value: parseFloat(row[1])
                            };
                        })
                        .filter(row => row !== null);
                    resolve(processedData);
                };
            } else {
                parseConfig.header = true;
                parseConfig.complete = results => {
                    const processedData = results.data
                        .map(row => {
                            const cityName = standardizeCityName(row['城市']);
                            if (!cityName) return null;
                            return {
                                ...row,
                                '城市': cityName
                            };
                        })
                        .filter(row => row !== null);
                    resolve(processedData);
                };
            }

            Papa.parse(content, parseConfig);
        });
    } catch (error) {
        console.error(`Error reading file ${filePath}:`, error);
        return null;
    }
}

// [findCSVFiles 和 mergeHousingData 函数保持不变]
async function findCSVFiles(baseDir) {
    const csvFiles = [];
    
    async function scanDirectory(dir) {
        const items = await fs.readdir(dir, { withFileTypes: true });
        
        for (const item of items) {
            const fullPath = path.join(dir, item.name);
            if (item.isDirectory()) {
                await scanDirectory(fullPath);
            } else if (item.name.endsWith('.csv') && item.name.startsWith('processed2_')) {
                csvFiles.push(fullPath);
            }
        }
    }
    
    await scanDirectory(baseDir);
    return csvFiles;
}

async function mergeHousingData(baseDir) {
    let mergedData = new Map();
    let cityVariants = new Map(); // 用于记录城市名称的变体

    try {
        console.log(`\nScanning directory: ${baseDir}`);
        const csvFiles = await findCSVFiles(baseDir);
        console.log(`\nFound ${csvFiles.length} CSV files:`);
        csvFiles.forEach(file => console.log(`- ${file}`));

        for (const filePath of csvFiles) {
            const fileName = path.basename(filePath);
            const fileNameParts = fileName.split('_');
            const yyyymm = fileNameParts[1];
            const tableType = fileNameParts[3].split('.')[0];
            const year = parseInt(yyyymm.substring(0, 4));
            const month = parseInt(yyyymm.substring(4, 6));

            const data = await readCSVFile(filePath, tableType);
            if (!data) continue;

            data.forEach(row => {
                const city = tableType === '1' || tableType === '2' ? row.city : row['城市'];
                const key = `${city}_${year}_${month}`;
                
                if (!mergedData.has(key)) {
                    mergedData.set(key, {
                        city,
                        year,
                        month,
                        new_house_price_index: null,
                        second_hand_price_index: null,
                        new_small_house_index: null,
                        new_medium_house_index: null,
                        new_large_house_index: null,
                        second_small_house_index: null,
                        second_medium_house_index: null,
                        second_large_house_index: null
                    });
                }

                const record = mergedData.get(key);

                switch (tableType) {
                    case '1':
                        record.new_house_price_index = row.value;
                        break;
                    case '2':
                        record.second_hand_price_index = row.value;
                        break;
                    case '3':
                        record.new_small_house_index = parseFloat(row['90m2及以下']);
                        record.new_medium_house_index = parseFloat(row['90-144m2']);
                        record.new_large_house_index = parseFloat(row['144m2以上']);
                        break;
                    case '4':
                        record.second_small_house_index = parseFloat(row['90m2及以下']);
                        record.second_medium_house_index = parseFloat(row['90-144m2']);
                        record.second_large_house_index = parseFloat(row['144m2以上']);
                        break;
                }
            });
        }

        const result = Array.from(mergedData.values())
            .sort((a, b) => {
                if (a.year !== b.year) return a.year - b.year;
                if (a.month !== b.month) return a.month - b.month;
                return a.city.localeCompare(b.city);
            });

        return result;
    } catch (error) {
        console.error('Error merging data:', error);
        return [];
    }
}

async function writeToCSV(data, outputPath) {
    try {
        // 数据处理
        const citiesData = new Map();
        data.forEach(row => {
            if (!citiesData.has(row.city)) {
                citiesData.set(row.city, []);
            }
            citiesData.get(row.city).push(row);
        });

        // 数据分析报告
        console.log('\n=== 数据质量报告 ===');
        
        // 异常检测
        const expectedRecords = 168;
        console.log('\n可能的异常数据:');
        citiesData.forEach((records, city) => {
            if (records.length !== expectedRecords) {
                console.log(`警告: ${city} 的数据条数(${records.length})异常，期望值为${expectedRecords}`);
                
                // 详细分析异常
                const years = Array.from(new Set(records.map(r => r.year))).sort();
                const missingMonths = [];
                for (let year = years[0]; year <= years[years.length-1]; year++) {
                    const monthsInYear = records.filter(r => r.year === year).map(r => r.month);
                    for (let month = 1; month <= 12; month++) {
                        if (!monthsInYear.includes(month)) {
                            missingMonths.push(`${year}-${month.toString().padStart(2, '0')}`);
                        }
                    }
                }
                if (missingMonths.length > 0) {
                    console.log(`  缺失月份: ${missingMonths.join(', ')}`);
                }
            }
        });

        // 城市列表
        const cities = Array.from(citiesData.keys()).sort();
        console.log('\n城市列表及统计:');
        cities.forEach((city, index) => {
            const records = citiesData.get(city);
            const years = Array.from(new Set(records.map(r => r.year))).sort();
            const recordsPerYear = {};
            years.forEach(year => {
                recordsPerYear[year] = records.filter(r => r.year === year).length;
            });

            console.log(`${(index + 1).toString().padStart(2, '0')}. ${city}`);
            console.log(`   数据条数: ${records.length}`);
            console.log(`   年份范围: ${years[0]} - ${years[years.length-1]}`);
            console.log(`   各年数据条数: ${Object.entries(recordsPerYear)
                .map(([year, count]) => `${year}年(${count}条)`)
                .join(', ')}`);
            console.log('');
        });

        // 写入数据
        const csv = Papa.unparse(data, {
            encoding: 'utf8'
        });
        await fs.writeFile(outputPath, '\ufeff' + csv, 'utf8');
        console.log(`\nData written to: ${outputPath}`);
        console.log(`总行数: ${data.length}`);
        console.log(`城市总数: ${cities.length}`);
        
    } catch (error) {
        console.error('Error writing CSV file:', error);
    }
}

async function main() {
    try {
        console.log(`Starting to process directory: ${dirPath}`);
        const mergedData = await mergeHousingData(dirPath);
        
        const outputPath = path.join(path.dirname(dirPath), 'merged_housing_data.csv');
        await writeToCSV(mergedData, outputPath);
        console.log('Processing completed.');
    } catch (error) {
        console.error('Error in main process:', error);
    }
}

main();

### Step 6: translate city names into English by dictionary

In [ ]:
# translate city names into English by dictionary
import pandas as pd

def convert_cities_to_english(input_file, output_file):
    # dictionary mapping between city names in English and Chinese
    city_mapping = {
        "三亚": "Sanya",
        "上海": "Shanghai",
        "丹东": "Dandong",
        "乌鲁木齐": "Urumqi",
        "九江": "Jiujiang",
        "兰州": "Lanzhou",
        "包头": "Baotou",
        "北京": "Beijing",
        "北海": "Beihai",
        "南京": "Nanjing",
        "南充": "Nanchong",
        "南宁": "Nanning",
        "南昌": "Nanchang",
        "厦门": "Xiamen",
        "合肥": "Hefei",
        "吉林": "Jilin",
        "呼和浩特": "Hohhot",
        "哈尔滨": "Harbin",
        "唐山": "Tangshan",
        "大理": "Dali",
        "大连": "Dalian",
        "天津": "Tianjin",
        "太原": "Taiyuan",
        "宁波": "Ningbo",
        "安庆": "Anqing",
        "宜昌": "Yichang",
        "岳阳": "Yueyang",
        "常德": "Changde",
        "平顶山": "Pingdingshan",
        "广州": "Guangzhou",
        "徐州": "Xuzhou",
        "惠州": "Huizhou",
        "成都": "Chengdu",
        "扬州": "Yangzhou",
        "无锡": "Wuxi",
        "昆明": "Kunming",
        "杭州": "Hangzhou",
        "桂林": "Guilin",
        "武汉": "Wuhan",
        "沈阳": "Shenyang",
        "泉州": "Quanzhou",
        "泸州": "Luzhou",
        "洛阳": "Luoyang",
        "济南": "Jinan",
        "济宁": "Jining",
        "海口": "Haikou",
        "深圳": "Shenzhen",
        "温州": "Wenzhou",
        "湛江": "Zhanjiang",
        "烟台": "Yantai",
        "牡丹江": "Mudanjiang",
        "石家庄": "Shijiazhuang",
        "福州": "Fuzhou",
        "秦皇岛": "Qinhuangdao",
        "蚌埠": "Bengbu",
        "襄阳": "Xiangyang",
        "西宁": "Xining",
        "西安": "Xi'an",
        "贵阳": "Guiyang",
        "赣州": "Ganzhou",
        "遵义": "Zunyi",
        "郑州": "Zhengzhou",
        "重庆": "Chongqing",
        "金华": "Jinhua",
        "银川": "Yinchuan",
        "锦州": "Jinzhou",
        "长春": "Changchun",
        "长沙": "Changsha",
        "青岛": "Qingdao",
        "韶关": "Shaoguan"
    }
    
    # read csv
    df = pd.read_csv(input_file)
    
    # replace
    df['city'] = df['city'].map(city_mapping)
    
    # save new csv
    df.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"done: {output_file}")


if __name__ == "__main__":
    input_file = "merged_housing_data.csv"
    output_file = "merged_housing_data_eng.csv"
    convert_cities_to_english(input_file, output_file)